<a href="https://colab.research.google.com/github/suzuki-hikaru/matplotlib-practice/blob/master/18118078_svm_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#機械学習授業第3回②
SVMを用いてワインの品質の多クラス分類をおこなってみてください！
特徴量は自分で選んでもいいですし、全部使ってもらっても構わないです。
特徴量を2つにした場合は2次元で表せるので、以下の可視化関数で境界を可視化することができます(svm_lecture.ipynbでも使ってます)
```python
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

def plot_decision_regions(X, y, classifier, test_idx=None, resolution=0.02):

    # setup marker generator and color map
    markers = ('s', 'x', 'o', '^', 'v')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    Z = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    Z = Z.reshape(xx1.shape)
    plt.contourf(xx1, xx2, Z, alpha=0.3, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0], 
                    y=X[y == cl, 1],
                    alpha=0.8, 
                    c=colors[idx],
                    marker=markers[idx], 
                    label=cl, 
                    edgecolor='black')

    # highlight test samples
    if test_idx:
        # plot all samples
        X_test, y_test = X[test_idx, :], y[test_idx]

        plt.scatter(X_test[:, 0],
                    X_test[:, 1],
                    c='',
                    edgecolor='black',
                    alpha=1.0,
                    linewidth=1,
                    marker='o',
                    s=100, 
                    label='test set')
```

# 提出課題解答欄
sklearn を用いる場合はモデル　.scote で正解率を算出すること
## 解答欄
### トレーニングデータの正解率：1.00

### テストデータの正解率：0.94

# データセット解説
## ワイン品質データセット(ロジスティック回帰のときと同じやつ)
アルコール濃度や色合いなどの特徴量からワインの

>**下記内容は事前にセルを用意している**
- データセットの読み込み
- トレーニングデータ・テストデータの分割 

In [ ]:
# インポート
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# 標準のデータセットから糖尿病データセットを読み込む
from sklearn import datasets
wine = datasets.load_wine()

# 読み込み
print('目的変数')
print(wine.target_names)
print('特徴量')
print(wine.feature_names)

目的変数
['class_0' 'class_1' 'class_2']
特徴量
['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']


In [ ]:
# データフレーム化
y = pd.Series(wine.target, name='Y')
X = pd.DataFrame(wine.data, columns=wine.feature_names)

# 全体のdaraframe作成
df = pd.concat([X, y], axis=1)
df.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,Y
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


In [ ]:
df.Y.unique()

array([0, 1, 2])

## 以降はsvm_lecture.ipynbを参考に?自由に分析してみてください

In [ ]:
# 準備
import seaborn as sns
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [ ]:
# テスト用に分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1, stratify=y)

# データを正規化する（後の講義で扱う）
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.fit_transform(X_test)

In [ ]:
from sklearn.svm import SVC

svm = SVC(kernel='linear', C=1, gamma=1, random_state=1)
svm.fit(X_train_std, y_train)

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='linear',
    max_iter=-1, probability=False, random_state=1, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
# 訓練データ
svm.score(X_train_std, y_train)

1.0

In [ ]:
# テストデータ
svm.score(X_test_std, y_test)

0.9444444444444444

参考ノートブックより、『パラメータCはロジスティック回帰のときに説明したものと同様に、正則化の強さを示すものです。gammaは大きいほど決定境界を複雑にする(直線じゃなくてギザギザ波波みたいな?)ものです。なので、Cと同様に大きくしすぎると訓練データに強く適合しようとするため、過学習してしまうことに繋がります。』

【gammaを大きくする】

In [ ]:
svm = SVC(kernel='linear', C=1, gamma=100, random_state=1)
svm.fit(X_train_std, y_train)

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=100, kernel='linear',
    max_iter=-1, probability=False, random_state=1, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
# 訓練データ
svm.score(X_train_std, y_train)


1.0

In [ ]:
# テストデータ
svm.score(X_test_std, y_test)

0.9444444444444444

過学習がない。変わらない?

【Cを大きくする】

In [ ]:
svm = SVC(kernel='linear', C=100, gamma=1, random_state=1)
svm.fit(X_train_std, y_train)

SVC(C=100, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='linear',
    max_iter=-1, probability=False, random_state=1, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
# 訓練データ
svm.score(X_train_std, y_train)

1.0

In [ ]:
# テストデータ
svm.score(X_test_std, y_test)

0.9444444444444444

Cを大きくしても変わらない。。